In [1]:
import pandas as pd
from pathlib import Path
import re

DATA_DIR = Path("..") / "data"
RESULTS_DIR = DATA_DIR / "results"
CLEAN_DIR = DATA_DIR / "clean"

RESULTS_DIR, list(RESULTS_DIR.glob("*.csv"))

(PosixPath('../data/results'), [])

In [2]:
import pandas as pd
from pathlib import Path
import re

DATA_DIR = Path("..") / "data"
RESULTS_DIR = DATA_DIR / "results"
CLEAN_DIR = DATA_DIR / "clean"

RESULTS_DIR, list(RESULTS_DIR.glob("*.csv"))

(PosixPath('../data/results'), [])

In [3]:
def parse_race_meta(meta_str: str):
    """
    Parse a meta line like:
    'Telenet Superprestige • Exact-Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED'
    or
    'Exact-Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED'

    Returns: (series_name, race_name, race_date, race_location)
    - series_name can be None for standalone races
    """
    if not isinstance(meta_str, str):
        return None, None, None, None

    parts = [p.strip() for p in meta_str.split("•") if p.strip()]
    
    series_name = None
    race_name = None
    race_date = None
    race_location = None

    # 4 parts: Series • Race name • Date • Location
    if len(parts) == 4:
        series_name, race_name, date_str, race_location = parts
    # 3 parts: Race name • Date • Location
    elif len(parts) == 3:
        race_name, date_str, race_location = parts
    else:
        # Fallback: treat the whole thing as race_name
        race_name = meta_str
        date_str = None
        race_location = None

    if 'date_str' in locals() and date_str:
        try:
            race_date = pd.to_datetime(date_str, format="%b %d %Y")
        except Exception:
            race_date = pd.to_datetime(date_str, errors="coerce")

    return series_name, race_name, race_date, race_location

In [4]:
def slugify(s):
    if not isinstance(s, str):
        return ""
    return re.sub(r"[^a-z0-9]+", "-", s.lower()).strip("-")

def make_race_id(series_name, race_name, race_date, location):
    date_part = race_date.strftime("%Y%m%d") if (race_date is not None and pd.notnull(race_date)) else "unknown"
    series_part = slugify(series_name) if series_name else "standalone"
    name_part = slugify(race_name)
    loc_part = slugify(location)
    return f"{date_part}_{series_part}_{name_part}_{loc_part}"

In [5]:
csv_path = sorted(RESULTS_DIR.glob("*.csv"))[0]
print("Using file:", csv_path)

df_raw = pd.read_csv(csv_path)
df_raw.head(5)

IndexError: list index out of range

In [6]:
import os
os.getcwd()

'/Users/marlex/projects/cyclocross-predictions'

In [7]:
from pathlib import Path

# Since your notebook is in the project root:
# /Users/marlex/projects/cyclocross-predictions
# and your data is in: /Users/marlex/projects/cyclocross-predictions/data
DATA_DIR = Path("data")
RESULTS_DIR = DATA_DIR / "results"
CLEAN_DIR = DATA_DIR / "clean"

print("DATA_DIR =", DATA_DIR.resolve())
print("RESULTS_DIR =", RESULTS_DIR.resolve())
print("CSV files found:", list(RESULTS_DIR.glob("*.csv")))

DATA_DIR = /Users/marlex/projects/cyclocross-predictions/data
RESULTS_DIR = /Users/marlex/projects/cyclocross-predictions/data/results
CSV files found: [PosixPath('data/results/UEC CycloCross •  European Championships • Nov 9 2025 • Middelkerke, BEL.csv'), PosixPath('data/results/X²O Badkamers Trofee •  Rapencross • Nov 2 2025 • Lokeren, BEL.csv'), PosixPath('data/results/X²O Badkamers Trofee • Koppenbergcross • Nov 1 2025 • Oudenaarde, BEL.csv'), PosixPath('data/results/Telenet Superprestige •  Niel • Nov 11 2025 • Niel, BEL.csv'), PosixPath('data/results/Exact Cross • Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED.csv'), PosixPath('data/results/Telenet Superprestige •  Ruddervoorde • Oct 19 2025 • Ruddervoorde, BEL.csv'), PosixPath('data/results/Exact Cross •  Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv'), PosixPath('data/results/Exact Cross • Berencross • Oct 4 2025 • Tielt (Meulebeke), BEL.csv'), PosixPath('data/results/Telenet Superprestige • Ove

In [8]:
csv_path = sorted(RESULTS_DIR.glob("*.csv"))[0]
print("Using file:", csv_path)

df_raw = pd.read_csv(csv_path)
df_raw.head(5)

Using file: data/results/Exact Cross •  Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv


,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points,Unnamed: 10
0,Men Elite,1,129684,Toon,Vandebosch,CRELAN-CORENDON,1:04:00,NaN,151.572152,135.875229,NaN
1,Men Elite,2,174032,Victor,Van de putte,DESCHACHT-HENS CX TEAM,1:04:01,NaN,170.054789,140.548828,NaN
2,Men Elite,3,144667,Niels,Vandeputte,NaN,1:04:01,NaN,130.398560,145.222426,NaN
3,Men Elite,4,181210,Wout,Janssen,NaN,1:04:02,NaN,181.522616,149.896025,NaN
4,Men Elite,5,153653,Cameron,Mason,SEVEN RACING,1:04:02,NaN,157.932693,154.569623,NaN


In [9]:
def parse_race_meta(meta_str: str):
    """
    Parse a meta line like:
    'Telenet Superprestige • Exact-Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED'
    or
    'Exact-Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED'

    Returns: (series_name, race_name, race_date, race_location)
    - series_name can be None for standalone races
    """
    if not isinstance(meta_str, str):
        return None, None, None, None

    parts = [p.strip() for p in meta_str.split("•") if p.strip()]
    
    series_name = None
    race_name = None
    race_date = None
    race_location = None

    # 4 parts: Series • Race name • Date • Location
    if len(parts) == 4:
        series_name, race_name, date_str, race_location = parts
    # 3 parts: Race name • Date • Location
    elif len(parts) == 3:
        race_name, date_str, race_location = parts
    else:
        # Fallback: treat the whole thing as race_name
        race_name = meta_str
        date_str = None
        race_location = None

    if 'date_str' in locals() and date_str:
        try:
            race_date = pd.to_datetime(date_str, format="%b %d %Y")
        except Exception:
            race_date = pd.to_datetime(date_str, errors="coerce")

    return series_name, race_name, race_date, race_location

In [10]:
def slugify(s):
    if not isinstance(s, str):
        return ""
    return re.sub(r"[^a-z0-9]+", "-", s.lower()).strip("-")

def make_race_id(series_name, race_name, race_date, location):
    date_part = race_date.strftime("%Y%m%d") if (race_date is not None and pd.notnull(race_date)) else "unknown"
    series_part = slugify(series_name) if series_name else "standalone"
    name_part = slugify(race_name)
    loc_part = slugify(location)
    return f"{date_part}_{series_part}_{name_part}_{loc_part}"

In [11]:
csv_path = sorted(RESULTS_DIR.glob("*.csv"))[0]
print("Using file:", csv_path)

df_raw = pd.read_csv(csv_path)
df_raw.head(5)

Using file: data/results/Exact Cross •  Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv


,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points,Unnamed: 10
0,Men Elite,1,129684,Toon,Vandebosch,CRELAN-CORENDON,1:04:00,NaN,151.572152,135.875229,NaN
1,Men Elite,2,174032,Victor,Van de putte,DESCHACHT-HENS CX TEAM,1:04:01,NaN,170.054789,140.548828,NaN
2,Men Elite,3,144667,Niels,Vandeputte,NaN,1:04:01,NaN,130.398560,145.222426,NaN
3,Men Elite,4,181210,Wout,Janssen,NaN,1:04:02,NaN,181.522616,149.896025,NaN
4,Men Elite,5,153653,Cameron,Mason,SEVEN RACING,1:04:02,NaN,157.932693,154.569623,NaN


In [12]:
meta_str = str(df_raw.loc[0, "Category Name"])
series_name, race_name, race_date, race_location = parse_race_meta(meta_str)

series_name, race_name, race_date, race_location

(None, 'Men Elite', None, None)

In [13]:
import pandas as pd
import re
from pathlib import Path

def parse_race_meta_from_filename(path: Path):
    """
    Parse a filename like:
    'Exact Cross • Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv'

    Returns: (series_name, race_name, race_date, race_location)
    """
    stem = path.stem  # strip ".csv"
    parts = [p.strip() for p in stem.split("•") if p.strip()]

    series_name = None
    race_name = None
    race_date = None
    race_location = None

    # 4 parts: Series • Race name • Date • Location
    if len(parts) == 4:
        series_name, race_name, date_str, race_location = parts
    # 3 parts: Race name • Date • Location (no series)
    elif len(parts) == 3:
        race_name, date_str, race_location = parts
    else:
        # Fallback: treat whole stem as race_name
        race_name = stem
        date_str = None
        race_location = None

    if 'date_str' in locals() and date_str:
        try:
            race_date = pd.to_datetime(date_str, format="%b %d %Y")
        except Exception:
            race_date = pd.to_datetime(date_str, errors="coerce")

    return series_name, race_name, race_date, race_location

def slugify(s):
    if not isinstance(s, str):
        return ""
    return re.sub(r"[^a-z0-9]+", "-", s.lower()).strip("-")

def make_race_id(series_name, race_name, race_date, location):
    date_part = race_date.strftime("%Y%m%d") if (race_date is not None and pd.notnull(race_date)) else "unknown"
    series_part = slugify(series_name) if series_name else "standalone"
    name_part = slugify(race_name)
    loc_part = slugify(location)
    return f"{date_part}_{series_part}_{name_part}_{loc_part}"

In [14]:
test_path = Path("data/results/Exact Cross • Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv")
parse_race_meta_from_filename(test_path)

('Exact Cross',
 'Robotland Cyclocross Essen',
 Timestamp('2025-10-18 00:00:00'),
 'Essen, BEL')

In [15]:
DATA_DIR = Path("data")
RESULTS_DIR = DATA_DIR / "results"
CLEAN_DIR = DATA_DIR / "clean"

all_results = []

for csv_path in sorted(RESULTS_DIR.glob("*.csv")):
    print(f"Processing {csv_path.name} ...")

    # Parse meta from filename
    series_name, race_name, race_date, race_location = parse_race_meta_from_filename(csv_path)
    race_id = make_race_id(series_name, race_name, race_date, race_location)

    # Read CSV
    df = pd.read_csv(csv_path)

    # Drop empty "Unnamed" columns (like Unnamed: 10)
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

    # Add metadata columns
    df["series_name"]   = series_name
    df["race_name"]     = race_name
    df["race_date"]     = race_date
    df["race_location"] = race_location
    df["race_id"]       = race_id

    # Build full rider name
    df["rider_name"] = (
        df["First Name"].fillna("").astype(str).str.strip()
        + " "
        + df["Last Name"].fillna("").astype(str).str.strip()
    ).str.strip()

    # Make Place numeric
    df["Place"] = pd.to_numeric(df["Place"], errors="coerce")

    all_results.append(df)

results_all = pd.concat(all_results, ignore_index=True)

results_all.head(10)

Processing Exact Cross •  Robotland Cyclocross Essen • Oct 18 2025 • Essen, BEL.csv ...
Processing Exact Cross • Berencross • Oct 4 2025 • Tielt (Meulebeke), BEL.csv ...
Processing Exact Cross • Internationale Cyclocross Heerderstrand • Oct 25 2025 • Heerde, NED.csv ...
Processing Telenet Superprestige •  Niel • Nov 11 2025 • Niel, BEL.csv ...
Processing Telenet Superprestige •  Ruddervoorde • Oct 19 2025 • Ruddervoorde, BEL.csv ...
Processing Telenet Superprestige • Overijse • Oct 26 2025 • Overijse, BEL.csv ...
Processing UEC CycloCross •  European Championships • Nov 9 2025 • Middelkerke, BEL.csv ...
Processing X²O Badkamers Trofee •  Rapencross • Nov 2 2025 • Lokeren, BEL.csv ...
Processing X²O Badkamers Trofee • Koppenbergcross • Nov 1 2025 • Oudenaarde, BEL.csv ...


,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points,series_name,race_name,race_date,race_location,race_id,rider_name
0,Men Elite,1,129684,Toon,Vandebosch,CRELAN-CORENDON,1:04:00,NaN,151.572152,135.875229,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Toon Vandebosch
1,Men Elite,2,174032,Victor,Van de putte,DESCHACHT-HENS CX TEAM,1:04:01,NaN,170.054789,140.548828,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Victor Van de putte
2,Men Elite,3,144667,Niels,Vandeputte,NaN,1:04:01,NaN,130.398560,145.222426,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Niels Vandeputte
3,Men Elite,4,181210,Wout,Janssen,NaN,1:04:02,NaN,181.522616,149.896025,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Wout Janssen
4,Men Elite,5,153653,Cameron,Mason,SEVEN RACING,1:04:02,NaN,157.932693,154.569623,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Cameron Mason
5,Men Elite,6,36277,Laurens,Sweeck,CRELAN-CORENDON,1:04:04,NaN,135.263615,159.243221,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Laurens Sweeck
6,Men Elite,7,76451,Kevin,Suarez fernandez,NESTA - MMR CYLING TEAM,1:04:43,NaN,188.379424,163.916820,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Kevin Suarez fernandez
7,Men Elite,8,196599,Kenay,De moyer,PAUWELS SAUZEN - ALTEZ INDUSTRIEBOUW CYCLING T...,1:05:03,NaN,184.219719,168.590418,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Kenay De moyer
8,Men Elite,9,22212,Tom,Meeusen,NaN,1:05:05,NaN,183.164329,173.264016,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Tom Meeusen
9,Men Elite,10,218295,Mats,Vanden eynde,BALOISE GLOWI LIONS,1:05:09,NaN,229.291239,177.937615,Exact Cross,Robotland Cyclocross Essen,2025-10-18,"Essen, BEL",20251018_exact-cross_robotland-cyclocross-esse...,Mats Vanden eynde


In [16]:
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
output_path = CLEAN_DIR / "results_all.csv"
results_all.to_csv(output_path, index=False)

output_path

PosixPath('data/clean/results_all.csv')